# Intro

This is mainly used as a debut and to track evolution.

In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
import cv2 
import os

In [3]:
train_path = "../data/raw/train"
metadata_path = "../data/raw/metadata.csv"

df = pd.read_csv(metadata_path)
df = df.set_index("split").loc['train'].reset_index()
df


,split,image_id,sat_image_path,mask_path
0,train,100034,train/100034_sat.jpg,train/100034_mask.png
1,train,100081,train/100081_sat.jpg,train/100081_mask.png
2,train,100129,train/100129_sat.jpg,train/100129_mask.png
3,train,100703,train/100703_sat.jpg,train/100703_mask.png
4,train,100712,train/100712_sat.jpg,train/100712_mask.png
...,...,...,...,...
6221,train,999431,train/999431_sat.jpg,train/999431_mask.png
6222,train,999531,train/999531_sat.jpg,train/999531_mask.png
6223,train,999667,train/999667_sat.jpg,train/999667_mask.png
6224,train,999764,train/999764_sat.jpg,train/999764_mask.png


In [4]:
def load_data(split_path, metadata_path, resize_shape=None):

    images = []
    masks = [] 
    
    split_name = split_path.split("/")[-1]
    start_path = split_path[:-len(split_name)]

    meta = pd.read_csv(metadata_path)
    meta = df.set_index("split").loc[split_name].reset_index()

    for i in range(len(meta)):
        im = start_path + meta.loc[i]['sat_image_path']
        ma = start_path + meta.loc[i]['mask_path']
        if not os.access(im, os.R_OK) and not os.access(ma, os.R_OK):
            raise FileNotFoundError(f"Either one of the following files is not found: {im} or {ma}")
        
        try:
            im = cv2.imread(im)
            ma = cv2.imread(ma, cv2.IMREAD_GRAYSCALE)

            if resize_shape:
                im = cv2.resize(im, resize_shape)
                ma = cv2.resize(ma, resize_shape)

            im = im / 255.0
            ma = ma / 255.0
        except:
            print("issue on read/transforms")
            pass
        
        images.append(im)
        masks.append(ma)

    return images, masks


images, masks =  load_data(train_path, metadata_path, resize_shape=(128,128))

In [ ]:
from tensorflow.keras import layers


def unet(input_shape):

    # Input
    inputs = layers.Input(shape=input_shape)

    # Encoder parameters
    encoder_parameters = {
        "filter" : [64, 64],
        "kernel_size": [(3,3), (3,3)],
        "padding" : ['same', 'same'],
        "pool_size" : [(2,2)],
        "activation": [],
        "activation"
    }
    # Encoder
    conv1_1 = layers.Conv2D(encoder_parameters['filter'][0], encoder_parameters['kernel_size'][0], activation='relu', padding=encoder_parameters['padding'][0])(inputs)
    conv1_2 = layers.Conv2D(encoder_parameters['filter'][0], encoder_parameters['kernel_size'][0], activation='relu', padding=encoder_parameters['padding'][0])(conv1_1)
    pool1   = layers.MaxPooling2D(encoder_parameters['pool_size'][0]) (conv1_2)
    # Bottleneck
    
    # Decoder
    